In [1]:
import json
import pandas
import numpy
import re

In [2]:
with open('output.txt') as f:
    file_content = f.read()

In [3]:
lines = file_content.split('\n')

In [4]:
print(lines[0])
print(lines[1])

00:59:59.974915 IP 117.104.234.209.50853 > 203.80.133.159.40014: UDP, length 171
00:59:59.974933 IP 74.67.249.149.46974 > 163.29.128.115.465: Flags [S], seq 251008648, win 1024, options [mss 1460], length 0


In [6]:

def split(s):
    parts = []
    bracket_level = 0
    current = []
    # trick to remove special-case of trailing chars
    for c in (s + ","):
        if c == "," and bracket_level == 0:
            parts.append("".join(current))
            current = []
        else:
            if c == "[":
                bracket_level += 1
            elif c == "]":
                bracket_level -= 1
            current.append(c)
    return parts

In [21]:
UDP = []
TCP = []
OTHER = []
ERRORS = []
GENERAL = []

for line in lines:
    const_split = line.split(' ')
    
    if len(const_split) < 5:
        continue
    
    entry = {}
    entry['ts'] = const_split[0]
    entry['ip_type'] = const_split[1]
    entry['source_ip'] = 'NA'
    entry['source_port'] = 'NA'
    entry['dest_ip'] = 'NA'
    entry['dest_port'] = 'NA'
    
    if entry['ip_type'] == 'IP':
        if len(const_split[2].split('.')) == 4:
            entry['source_ip'] = const_split[2]
            entry['source_port'] = 'NA'
            
        else:
            ip_split_source = const_split[2].rsplit('.', 1)
            entry['source_ip'] = ip_split_source[0]
            entry['source_port'] = ip_split_source[1]
            
        #Then destination    
        if len(const_split[4][:-1].split('.')) == 4:
            entry['dest_ip'] = const_split[4][:-1]
            entry['dest_port'] = 'NA'
            
        else:
            ip_split_dest = const_split[4][:-1].rsplit('.', 1)
            entry['dest_ip'] = ip_split_dest[0]
            entry['dest_port'] = ip_split_dest[1]
        
    else: #IPv6
        #Source first
        if len(const_split[2].rsplit('.', 1)) == 1:
            entry['source_ip'] = const_split[2]
            entry['source_port'] = 'NA'
        else:
            entry['source_ip'] = const_split[2].rsplit('.', 1)[0]
            entry['source_port'] = const_split[2].rsplit('.', 1)[1]
            
        #Then destination
        if len(const_split[4][:-1].rsplit('.', 1)) == 1:
            entry['dest_ip'] = const_split[4][:-1]
            entry['dest_port'] = 'NA'
            
        else:
            ip_split_dest = const_split[4][:-1].rsplit('.', 1)
            entry['dest_ip'] = ip_split_dest[0]
            entry['dest_port'] = ip_split_dest[1]
        
    
    second_half = const_split[5:]
    
    if len(second_half) == 0:
        continue
    
    protocol = second_half[0]
    
    
    if protocol == 'UDP,':
        protocol = 'UDP'
        
    if protocol != 'UDP':
        protocol = 'UNKNOWN'
        
        
    if second_half[0] == 'Flags':
        protocol = 'TCP'
        
    if second_half[0] == 'ICMP' or second_half[0] == 'ICMP6':
        protocol = 'ICMP'
        
        
    entry['protocol'] = protocol
    
    if protocol == 'UDP':
        second_half = second_half[1:]
        # Either successful with   length or bad length returned 
        entry['result'] = 'Success'
        entry['expected_length'] = -1
        entry['length'] = -1
        
        if second_half[0] == 'bad':
            entry['result'] = 'Failure'
            entry['length'] = second_half[2]
            entry['expected_length'] = second_half[4]
            
        else:
            entry['length'] = second_half[1]
            entry['expected_length'] = second_half[1]
            
            
        UDP.append(entry)
        GENERAL.append(entry)
        
    
    elif protocol =='TCP':
        ln = " "
        for token in second_half:
            ln += token + " "
            
        kvs = {}
        entry['result'] = 'Success'
        entry['length'] = -1
        entry['expected_length'] = -1
    
        splits = split(ln)
        for kv_pair in splits:
            pair = kv_pair.strip().split( ' ', 1)
            kvs[pair[0]] = pair[1]
            
        
        if 'length' in kvs:
            entry['length'] = kvs['length']
            entry['expected_length'] = kvs['length']
            
        else:
            entry['result'] = 'Failure'
            
            
        TCP.append(entry)
        GENERAL.append(entry)
        
    
    elif protocol == 'ICMP':
        ln = " "
        for token in second_half:
            ln += token + " "
            
        ln = ln.strip()
        
        em = ln.split(' ',1)[1]
        
        tokens = em.split(',')
        entry['error_message'] = tokens[0].strip()
        
        for token in tokens[1:]:
            kv = token.strip().split(' ',1)
            entry[kv[0]] = kv[1]
        
        
        ERRORS.append(entry)
        
    
    else:
        ln = " "
        for token in second_half:
            ln += token + " "
            
        ln = ln.strip()
        
        entry['details'] = ln
        
        OTHER.append(entry)

In [8]:
print(len(GENERAL))

1480282


In [22]:
with open('unified.json', 'w+') as f:
    f.write(json.dumps(GENERAL[0:200000]))

In [24]:
from json_excel_converter import Converter 
from json_excel_converter.xlsx import Writer
conv = Converter()
conv.convert(GENERAL[0:200000], Writer(file='/Users/dhruvnandakumar/Desktop/CoE/MAWI_generalized.xlsx'))